In [5]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
import requests
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-03-11_11:49
E:\RIS\Staff Folders\Samuel\Requests\SH\CH_Request_Data_30620


In [2]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')


Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-10 17:02:02


In [9]:
#today = datetime.datetime.now().strftime('%Y_%m_%d')

#speed_humps_url = 'https://data.cityofnewyork.us/api/views/9n6h-pt9g/rows.csv'
#humps_file = requests.get(speed_humps_url)

#open(r'\\dot55fp05\BOTPLAN\RIS\Requests\Citywide\City_Hall\SchoolIntersectionTracker\SRTS_Data\opendata_srts_{0}.csv'.format(today), 'wb').write(humps_file.content)

#open_speed_humps = pd.DataFrame.from_csv(r'\\dot55fp05\BOTPLAN\RIS\Requests\Citywide\City_Hall\SchoolIntersectionTracker\SRTS_Data\opendata_srts_{0}.csv'.format(today))
#open_speed_humps.head(1)

### Reading in SRTS Speed Humps Table

In [3]:
sh_data = pd.read_excel(r'Data\SRTS_CBapproved_since2018.xlsx')

### Creating Speed Humps Ids for Each Distinct Project

In [4]:
sh_data['srid'] = map(lambda x: str(x)[-5::],sh_data.ProjectCode) 

sh_data['SID'] = map(lambda x: str(x), sh_data.SegmentID)

sh_data['speed_hump_id'] = sh_data['srid'] + sh_data['SID']

### Writing Table To PG

In [5]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-03-10 17:02:05


In [6]:
d = cdb.dataframe_to_table(sh_data, 'SRTS_30620', schema= 'working', overwrite = True)

Reading data into Database



233it [00:06, 37.38it/s]



233 rows added to working.SRTS_30620



### Creating Table With All Nearest to School and Treatment Nodeids and Treatment Details 

In [7]:
cdb.query("""

DROP TABLE IF EXISTS working.ch_speed_humps;

CREATE TABLE working.ch_speed_humps AS

WITH data AS(

SELECT DISTINCT ch_sh.*, lion.*
FROM working.SRTS_30620 ch_sh
LEFT JOIN (SELECT l.segmentid, l.masteridfrom, l.masteridto, l.geom segment_geom, nlu.*
       FROM archive."18d.2019-11-13_lion" l
       JOIN public.ch_schools_mft_node_lookup nlu
       on l.mft = nlu.mft) lion 
ON ch_sh.segmentid::int = lion.segmentid::int
WHERE SegmentStatus = 1
ORDER BY lion.mft, speed_hump_id, dist
)                                                                                                                                                                                                                                                                       


--Nodes with the minimum distance from schools 
,data2 AS(
SELECT data.*, d1.min_school_node_dist FROM(
SELECT speed_hump_id, min(dist) min_school_node_dist
FROM data 
GROUP BY speed_hump_id
) d1
JOIN data 
on  data.speed_hump_id = d1.speed_hump_id and data.dist = d1.min_school_node_dist
)


--Grabbing all distances of nodeid to treatment
,data3 AS(
--Based on the masterid, selecting m
SELECT DISTINCT d1.*, d2.min_school_node_dist, ST_DISTANCE(d1.geom,d1.segment_geom) treat_dist
FROM data2 d2
LEFT JOIN data d1
ON d2.masterid = d1.masterid
)


--Grabbing all minimum distances of nodeid to treatment
,data4 AS (
SELECT d3.speed_hump_id, d3.CBAcceptedProposalDate::date speed_hump_approval, d3.numsrproposed number_humps, d3.masterid, d3.nodeid, d3.treat_dist, d3.dist, d3.on_st on_street, d3.cross_st cross_street, d3.school_list, d3.school_id_list, d3.address_list  
FROM (  SELECT d_3.*, data3.masterid
    FROM (SELECT d3.speed_hump_id, min(treat_dist) treat_dist 
    FROM data3 d3
    GROUP BY d3.speed_hump_id
    ) d_3
    LEFT JOIN data3  
    ON d_3.speed_hump_id = data3.speed_hump_id and d_3.treat_dist  = data3.treat_dist  
) treat_data
JOIN data3 d3
ON treat_data.speed_hump_id = d3.speed_hump_id and treat_data.masterid = d3.masterid and treat_data.treat_dist = d3.treat_dist
)


SELECT * FROM data4

"""
)

Query run in 1 seconds


### Updating City Hall Tracking Master Table 

In [8]:
cdb.query("""

--From the speed hump data, min(speed_hump_id) in the case where there are multiple speed hump ids assigned to the same segmentid

INSERT INTO public.ch_schools_output as so (nodeid, dist, on_street, cross_street, school_list, school_id_list, address_list, speed_hump_id, speed_hump_approval, number_humps)
SELECT DISTINCT nodeid,dist,on_street, cross_street, school_list, school_id_list, address_list, min(speed_hump_id) speed_hump_id, speed_hump_approval, number_humps
FROM working.ch_speed_humps 
GROUP BY nodeid,dist,on_street, cross_street, school_list, school_id_list, address_list, speed_hump_approval, number_humps
ON CONFLICT (nodeid)
DO UPDATE 
    SET speed_hump_id = excluded.speed_hump_id, speed_hump_approval = excluded.speed_hump_approval, number_humps = excluded.number_humps
        WHERE so.speed_hump_id is null


""")

Query run in 8000 microseconds


### Speed Hump Ids that dont make it to the City Hall Tracking Master Table 
#### Because details for that node are already populated

In [9]:
dump = cdb.dfquery("""

SELECT ch_sh.nodeid, ch_sh.dist, ch_sh.on_street, ch_sh.cross_street, ch_sh.school_list, ch_sh.school_id_list, ch_sh.address_list, ch_sh.speed_hump_id, ch_sh.speed_hump_approval, ch_sh.number_humps
FROM working.ch_speed_humps ch_sh
LEFT JOIN public.ch_schools_output so
ON ch_sh.speed_hump_id = so.speed_hump_id
WHERE so.speed_hump_id is null

""")

In [10]:
dump.to_csv('dumped.csv', index = False)